Reason and Act, its core innovation is the way it enables an agent to dynamically reason about a problem, act on its reasoning, observe the outcome and then reason again

The ReAct architecture is a design pattern where an agent interleaves reasoning steps with actions. Instead of planning all its steps upfront, the agent generates a thought about its immediate next step, takes an action (like calling a tool), observes the result, and then uses that new information to generate its next thought and action. This creates a dynamic and adaptive loop.



## High-level Workflow
###### Receive Goal: The agent is given a complex task.

###### Think (Reason): The agent generates an internal thought, such as: "To answer this, I first need to find piece of information X."

###### Act: Based on its thought, the agent executes an action, typically calling a tool (e.g., search_api('X')).


###### Observe: The agent receives the result from the tool.

###### Repeat: The agent incorporates the observation into its context and returns to step 2, generating a new thought (e.g., "Okay, now that I have X, I need to use it to find Y."). This loop continues until the overall goal is satisfied.

In [1]:

!pip3 install -q -U langchain-openai langchain langgraph rich python-dotenv tavily-python


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import os
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from pydantic import BaseModel,Field

from langgraph.graph import StateGraph,END
from langgraph.graph.message import AnyMessage,add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from rich.console import Console
from rich.markdown import Markdown

os.environ['TAVILY_API_KEY']=""
os.environ['LANGCHAIN_API_KEY'] = ''

os.environ['OPENAI_API_KEY'] = ''



In [11]:
from typing import TypedDict

console=Console()

#Defining state fro our graphs
class AgentState(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

#Define the llm tool
search_tool=TavilySearchResults(max_results=2,name="web-search-tool")
llm=ChatOpenAI(model='gpt-4o-mini',temperature=0)
llm_with_tools=llm.bind_tools([search_tool])

def basic_agent_node(state:AgentState):
    console.print("---Bsic agent Thinking")
    #Note we provide a system prompt yo encouraeg it to answer directly after one tool call
    system_prompt="You are a helpful assistant.You have access to a web search tool. Answer the user's question based on the tool's results. You must provide a final answer after one tool call"
    messages=[("system",system_prompt)]+state["messages"]
    response=llm_with_tools.invoke(messages)
    return {"messages":[response]}
    

#Define the basic, linear growth
basic_graph_builder=StateGraph(AgentState)
basic_graph_builder.add_node("agent",basic_agent_node)
basic_graph_builder.add_node("tools",ToolNode([search_tool]))

basic_graph_builder.set_entry_point("agent")
#After the agent it can only go to tools and after toos, it must end.
basic_graph_builder.add_conditional_edges("agent",tools_condition,{'tools':'tools',"__end__":"__end__"})
basic_graph_builder.add_edge('tools',END)

basic_tool_agent_app=basic_graph_builder.compile()

print("BAsic single shot tool-using agent compiled succesfully")


BAsic single shot tool-using agent compiled succesfully


In [12]:
multi_step_query="Who is the CEO of the company that created sci-fi movue 'Dune', and what was the budget for that company's most recent film?"

console.print(f"[bold yellow ] Testing BASIC agent on a multi-step query:[/bold yellow]'{multi_step_query}'\n")

basic_agent_output=basic_tool_agent_app.invoke({"messages":[("user",multi_step_query)]})

console.print("\n ---[bold red] Final Output from Basic Agent[/bold red]---")
console.print(Markdown(basic_agent_output['messages'][-1].content))

 Testing BASIC agent on a multi-step query:'Who is the CEO of the company that created sci-fi movue 'Dune', and 
what was the budget for that company's most recent film?'

---Bsic agent Thinking

--- Final Output from Basic Agent---

[{"title": "Jason Clodfelter Named President, TV Legendary Entertainment", "url":                                  
"https://deadline.com/2023/08/jason-clodfelter-named-president-tv-legendary-entertainment-1235453231/", "content": 
"“We are pleased to welcome Jason to the Legendary team, where he will be a tremendous asset as we continue to grow
our television business with compelling content that resonates with our audiences worldwide,” said Legendary CEO   
Josh Grode. “His creative instincts, understanding of IP and ability to work with the creative community are key   
strengths that he brings to our television department and complements Carmi Zlotnik and our other senior           
executives. [...] Skip to main content\n\n# Sony TV Veteran Jason Clodfelter Named President Of Television At      
Legendary Entertainment\n\nLegendary Entertainment has a new President of Television.\n\nThe studio has hired Sony 
Pictures Television Studios Co‐President Jason Clodfelter in the role. [...] Jason Clodfelter, Co-President of SPT 
Studios, has decided to leave the company to take on a new role as President of Television at Legendary            
Entertainment. A longtime member of the Sony Pictures family, Jason’s move marks a change for our senior leadership
team, but we fully support his decision and wish him success as he embarks on this exciting new chapter in his     
career. Our colleagues in the motion picture group have a great partnership with Legendary to market and distribute
their films,", "score": 0.7559439}, {"title": "Legendary Entertainment Names Jason Clodfelter President of ...",   
"url":                                                                                                             
"https://variety.com/2023/tv/news/legendary-entertainment-jason-clodfelter-president-of-television-1235686141/",   
"content": "# Legendary Entertainment Names Jason Clodfelter President of Television\n\nLongtime Sony Pictures     
Entertainment executive Jason Clodfelter has been appointed president of television at Legendary Entertainment. The
announcement was made on Wednesday by Legendary CEO, Josh Grode.", "score": 0.74961954}]

Building ReAct Agent Graph

In [13]:
def react_agent_node(state: AgentState):
    console.print("----REACT Agent: Thinking ... ")
    response=llm_with_tools.invoke(state['messages'])
    return {'messages':[response]}

#The tool node is same as before
react_tool_node=ToolNode([search_tool])

#The router is also the same logic
def react_router(state:AgentState):
    last_message=state['messages'][-1]
    if last_message.tool_calls:
        console.print("--- ROUTER: Decision is to call a tool")
        return "tools"
    console.print("--- ROUTER: Decision is to finish")
    return "__end__"

# Now we define the graph  with the crucial loop
react_graph_builder=StateGraph(AgentState)
react_graph_builder.add_node("agent",react_agent_node)
react_graph_builder.add_node("tools",react_tool_node)

react_graph_builder.set_entry_point("agent")
react_graph_builder.add_conditional_edges("agent",react_router,{"tools":"tools","__end__":"__end__"})

#This is the key difference: the edges goes from tools BACK to agent
react_graph_builder.add_edge("tools","agent")

react_agent_app=react_graph_builder.compile()
print("ReAct Agent compiled successfully with a reasoning loop")



ReAct Agent compiled successfully with a reasoning loop


In [15]:
console.print(f"[bold green] Testing ReAct agent on the same multi-step query[/bold green] '{multi_step_query}'\n")

final_react_output=None
for chunk in react_agent_app.stream({"messages":[("user",multi_step_query)]},stream_mode="values"):
    final_react_output=chunk
    console.print(f"---[bold purple] Current State [/bold purple]---")
    chunk['messages'][-1].pretty_print()
    console.print("\n")

console.print("\n--- [bold green]Final Output from ReAct Agent[/bold green] ---")
console.print(Markdown(final_react_output['messages'][-1].content))

 Testing ReAct agent on the same multi-step query 'Who is the CEO of the company that created sci-fi movue 'Dune', 
and what was the budget for that company's most recent film?'

--- Current State ---

================================ Human Message =================================

Who is the CEO of the company that created sci-fi movue 'Dune', and what was the budget for that company's most recent film?


----REACT Agent: Thinking ...

--- ROUTER: Decision is to call a tool

...

--- ROUTER: Decision is to finish

--- Current State ---

================================== Ai Message ==================================

The CEO of Legendary Entertainment, the company that created the sci-fi movie "Dune," is **Josh Grode**. He has held this position since December 2017.

The budget for Legendary Entertainment's most recent film, "Dune: Part Two," was **$190 million**.


--- Final Output from ReAct Agent ---

The CEO of Legendary Entertainment, the company that created the sci-fi movie "Dune," is Josh Grode. He has held   
this position since December 2017.                                                                                 

The budget for Legendary Entertainment's most recent film, "Dune: Part Two," was $190 million.

In [16]:
class TaskEvaluation(BaseModel):
    """Schema for evaluating an agent's ability to complete a task."""
    task_completion_score: int = Field(description="Score 1-10 on whether the agent successfully completed all parts of the user's request.")
    reasoning_quality_score: int = Field(description="Score 1-10 on the logical flow and reasoning process demonstrated by the agent.")
    justification: str = Field(description="A brief justification for the scores.")

judge_llm = llm.with_structured_output(TaskEvaluation)

def evaluate_agent_output(query: str, agent_output: dict):
    trace = "\n".join([f"{m.type}: {m.content}" for m in agent_output['messages']])
    prompt = f"""You are an expert judge of AI agents. Evaluate the following agent's performance on the given task on a scale of 1-10. A score of 10 means the task was completed perfectly. A score of 1 means complete failure.
    
    **User's Task:**
    {query}
    
    **Full Agent Conversation Trace:**
    ```
    {trace}
    ```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Basic Agent's Output ---")
basic_agent_evaluation = evaluate_agent_output(multi_step_query, basic_agent_output)
console.print(basic_agent_evaluation.model_dump())

console.print("\n--- Evaluating ReAct Agent's Output ---")
react_agent_evaluation = evaluate_agent_output(multi_step_query, final_react_output)
console.print(react_agent_evaluation.model_dump())

--- Evaluating Basic Agent's Output ---

{
    'task_completion_score': 4,
    'reasoning_quality_score': 5,
    'justification': "The agent identified the CEO of Legendary Entertainment as Josh Grode, which is correct. 
However, it failed to provide the budget for the company's most recent film, which is a crucial part of the user's 
request. The reasoning quality is average as it presented relevant information but did not fully address the user's
question."
}

--- Evaluating ReAct Agent's Output ---

{
    'task_completion_score': 10,
    'reasoning_quality_score': 9,
    'justification': "The agent successfully identified the CEO of Legendary Entertainment as Josh Grode and 
provided the correct budget for 'Dune: Part Two' as $190 million. The information was accurate and directly 
addressed both parts of the user's request. The reasoning quality was high, but there was a slight lack of depth in
explaining the context of the budget or the film's performance, which is why it received a 9 instead of a 10."
}